# Discovering data

This notebook shows how to find out what data is available locally as well as on ESGF. It also shows how to download the data from ESGF.

In [1]:
from esmvalcore.config import CFG
from esmvalcore.dataset import Dataset

ERROR 1: PROJ: proj_create_from_database: Open of /home/manuel/micromamba/envs/esm/share/proj failed


Configure ESMValCore so it always searches the ESGF for data

In [2]:
CFG["search_data"] = "complete"
CFG.nested_update(
    {
        "projects": {
            "CMIP6": {
                "data": {
                    "intake-esgf": {
                        "type": "esmvalcore.io.intake_esgf.IntakeESGFDataSource",
                        "priority": 2,
                        "facets": {
                            "activity": "activity_drs",
                            "dataset": "source_id",
                            "ensemble": "member_id",
                            "exp": "experiment_id",
                            "institute": "institution_id",
                            "grid": "grid_label",
                            "mip": "table_id",
                            "project": "project",
                            "short_name": "variable_id",
                        },
                    },
                },
            },
        },
    },
)

We define a dataset template to search for all CMIP6 datasets that provide surface air temperature (tas) on a monthly resolution for the historical experiment. Note that ESMValCore uses its own names for the facets for a more uniform naming across different CMIP phases and other projects. The mapping to the facet names used on ESGF can be found in [esmvalcore.esgf.facets.FACETS](https://docs.esmvaltool.org/projects/esmvalcore/en/latest/api/esmvalcore.esgf.html#esmvalcore.esgf.facets.FACETS).

In [3]:
dataset_template = Dataset(
    short_name="tas",
    mip="Amon",
    project="CMIP6",
    exp="historical",
    dataset="*",
    institute="*",
    ensemble="*",
    grid="*",
)

Next, we use the `Dataset.from_files` method to build a list of datasets from the available files. This may take a while as searching the ESGF for many files is a bit slow. Because the search results are cached for a [configurable duration](https://docs.esmvaltool.org/projects/esmvalcore/en/latest/quickstart/configure.html#esgf-configuration), subsequent searches will be faster.

In [4]:
datasets = list(dataset_template.from_files())
print(f"Found {len(datasets)} datasets, showing the first 10:")
datasets[:10]

Found 906 datasets, showing the first 10:


[Dataset:
 {'dataset': 'TaiESM1',
  'project': 'CMIP6',
  'mip': 'Amon',
  'short_name': 'tas',
  'ensemble': 'r1i1p1f1',
  'exp': 'historical',
  'grid': 'gn',
  'institute': 'AS-RCEC'},
 Dataset:
 {'dataset': 'TaiESM1',
  'project': 'CMIP6',
  'mip': 'Amon',
  'short_name': 'tas',
  'ensemble': 'r2i1p1f1',
  'exp': 'historical',
  'grid': 'gn',
  'institute': 'AS-RCEC'},
 Dataset:
 {'dataset': 'AWI-CM-1-1-MR',
  'project': 'CMIP6',
  'mip': 'Amon',
  'short_name': 'tas',
  'ensemble': 'r1i1p1f1',
  'exp': 'historical',
  'grid': 'gn',
  'institute': 'AWI'},
 Dataset:
 {'dataset': 'AWI-CM-1-1-MR',
  'project': 'CMIP6',
  'mip': 'Amon',
  'short_name': 'tas',
  'ensemble': 'r2i1p1f1',
  'exp': 'historical',
  'grid': 'gn',
  'institute': 'AWI'},
 Dataset:
 {'dataset': 'AWI-CM-1-1-MR',
  'project': 'CMIP6',
  'mip': 'Amon',
  'short_name': 'tas',
  'ensemble': 'r3i1p1f1',
  'exp': 'historical',
  'grid': 'gn',
  'institute': 'AWI'},
 Dataset:
 {'dataset': 'AWI-CM-1-1-MR',
  'project': '

Let's look at the first dataset in more detail. We can print the facets describing the dataset:

In [5]:
dataset = datasets[0]
dataset

Dataset:
{'dataset': 'TaiESM1',
 'project': 'CMIP6',
 'mip': 'Amon',
 'short_name': 'tas',
 'ensemble': 'r1i1p1f1',
 'exp': 'historical',
 'grid': 'gn',
 'institute': 'AS-RCEC'}

and see what files are available:

In [6]:
dataset.files

[IntakeESGFDataset(name='CMIP6.CMIP.AS-RCEC.TaiESM1.historical.r1i1p1f1.Amon.tas.gn')]

Load a single file as `iris.cube.CubeList`:

In [7]:
cubes = dataset.files[0].to_iris()

Exception ignored in: <function tqdm.__del__ at 0x7f31abafa020>
Traceback (most recent call last):
  File "/home/manuel/micromamba/envs/esm/lib/python3.13/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/home/manuel/micromamba/envs/esm/lib/python3.13/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x7f31abafa020>
Traceback (most recent call last):
  File "/home/manuel/micromamba/envs/esm/lib/python3.13/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/home/manuel/micromamba/envs/esm/lib/python3.13/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
/home/manuel/micromamba/envs/esm/lib/python3.13/site-packages/intake_esgf/catalog.py:712: UserWarning: We co

DatasetLoadError: We were unable to load data for these keys:
- CMIP6.CMIP.AS-RCEC.TaiESM1.historical.r1i1p1f1.Amon.tas.gn
This could be for a few reasons:
- We failed to find file access information for these datasets.
- All the access links we found failed. The data nodes may be offline.
For more information, consult the session log 'print(cat.session_log())'.

`Dataset.from_files` can also handle derived variables properly:

In [8]:
dataset_template = Dataset(
    short_name="lwcre",
    mip="Amon",
    project="CMIP6",
    exp="historical",
    dataset="*",
    institute="*",
    ensemble="r1i1p1f1",
    grid="gn",
    derive=True,
    force_derivation=True,
)

In [9]:
datasets = list(dataset_template.from_files())
print(f"Found {len(datasets)} datasets, showing the first 10:")
datasets[:10]

Found 38 datasets, showing the first 10:


[Dataset:
 {'dataset': 'SAM0-UNICON',
  'project': 'CMIP6',
  'mip': 'Amon',
  'short_name': 'lwcre',
  'derive': True,
  'ensemble': 'r1i1p1f1',
  'exp': 'historical',
  'force_derivation': True,
  'grid': 'gn',
  'institute': 'SNU'},
 Dataset:
 {'dataset': 'AWI-ESM-1-1-LR',
  'project': 'CMIP6',
  'mip': 'Amon',
  'short_name': 'lwcre',
  'derive': True,
  'ensemble': 'r1i1p1f1',
  'exp': 'historical',
  'force_derivation': True,
  'grid': 'gn',
  'institute': 'AWI'},
 Dataset:
 {'dataset': 'CMCC-CM2-HR4',
  'project': 'CMIP6',
  'mip': 'Amon',
  'short_name': 'lwcre',
  'derive': True,
  'ensemble': 'r1i1p1f1',
  'exp': 'historical',
  'force_derivation': True,
  'grid': 'gn',
  'institute': 'CMCC'},
 Dataset:
 {'dataset': 'CESM2-WACCM',
  'project': 'CMIP6',
  'mip': 'Amon',
  'short_name': 'lwcre',
  'derive': True,
  'ensemble': 'r1i1p1f1',
  'exp': 'historical',
  'force_derivation': True,
  'grid': 'gn',
  'institute': 'NCAR'},
 Dataset:
 {'dataset': 'BCC-ESM1',
  'project': 'C

The facet `force_derivation=True` ensures variable derivation. If omitted and files that provide the variable `lwcre` without derivation are present, only those are returned.

If variable derivation is necessary (this will always be the case if `force_derivation=True` is used), the `files` attribute of the datasets may be empty. In this case, the input files of the input variables necessary for derivation can be accessed via the `Dataset.input_datasets` attribute:

In [10]:
dataset = datasets[0]
dataset.files

[]

In [12]:
for d in dataset.required_datasets:
    print(d["short_name"])
    print(d.files)

rlut
[LocalFile('/mnt/d/data/CMIP6/CMIP/SNU/SAM0-UNICON/historical/r1i1p1f1/Amon/rlut/gn/v20190323/rlut_Amon_SAM0-UNICON_historical_r1i1p1f1_gn_185001-185912.nc'), LocalFile('/mnt/d/data/CMIP6/CMIP/SNU/SAM0-UNICON/historical/r1i1p1f1/Amon/rlut/gn/v20190323/rlut_Amon_SAM0-UNICON_historical_r1i1p1f1_gn_186001-186912.nc'), LocalFile('/mnt/d/data/CMIP6/CMIP/SNU/SAM0-UNICON/historical/r1i1p1f1/Amon/rlut/gn/v20190323/rlut_Amon_SAM0-UNICON_historical_r1i1p1f1_gn_187001-187912.nc'), LocalFile('/mnt/d/data/CMIP6/CMIP/SNU/SAM0-UNICON/historical/r1i1p1f1/Amon/rlut/gn/v20190323/rlut_Amon_SAM0-UNICON_historical_r1i1p1f1_gn_188001-188912.nc'), LocalFile('/mnt/d/data/CMIP6/CMIP/SNU/SAM0-UNICON/historical/r1i1p1f1/Amon/rlut/gn/v20190323/rlut_Amon_SAM0-UNICON_historical_r1i1p1f1_gn_189001-189912.nc'), LocalFile('/mnt/d/data/CMIP6/CMIP/SNU/SAM0-UNICON/historical/r1i1p1f1/Amon/rlut/gn/v20190323/rlut_Amon_SAM0-UNICON_historical_r1i1p1f1_gn_190001-190912.nc'), LocalFile('/mnt/d/data/CMIP6/CMIP/SNU/SAM0-UNI